In [16]:
import pandas as pd
import json 
import pandas as pd 
import numpy as np
import collections
from collections import defaultdict
import re


In [17]:
class Domain_question_generation:
    
    def __init__(self, data):
        with open(data, 'r') as f:
            self.data = f.readlines()
        self.data = [json.loads(x) for x in self.data]
     
class Blocksworld(Domain_question_generation):
        
        def __init__(self, data):
            super().__init__(data)
        
        def create_breakdown_composite_qa_1(self,data,valid_action_sequences,index):    
            question_category = defaultdict(dict)
            solo_sequence = defaultdict(dict)
            for key, value in data.items():
                    for k, v in value.items():         
                        if value['part_of_plan?'] == True:            
                            if k == "fluents" and len(v) > 0: # Create a new defaultdict for each action
                                # exec_question = f"Can I execute the action of {key.split('_')[1].split('(')[0]}ing {key.split('(')[1].split(')')[0:1][0][0:2]} from {key.split('(')[1].split(')')[0:1][0][3:5]}?"
                                exec_question = f"Is the {key} executable?"
                                question_category['executable_questions_answers'][exec_question] = value['feasible?']
                                print
                                for fluent in v:
                                    if fluent.startswith('ontable'):
                                        block = fluent.split('(')[1].split(')')[0]
                                        question = "Which blocks are on the table?"
                                        if question in question_category['object_tracking']:
                                            question_category['object_tracking'][question].append(block)
                                        else:
                                            question_category['object_tracking'][question] = [block]    
                                        solo_sequence[valid_action_sequences[index]] = question_category
                                    elif fluent.startswith('clear'):
                                        block = fluent.split('(')[1].split(')')[0]
                                        question = "Which blocks are clear?"
                                        if question in question_category['object_tracking']:
                                            question_category['object_tracking'][question].append(block)
                                        else:
                                            question_category['object_tracking'][question] = [block] 
                                        solo_sequence[valid_action_sequences[index]] = question_category
                                    elif fluent.startswith('holding'):
                                        held_block = fluent.split('(')[1].split(')')[0]
                                        question = "Which block am I holding?"
                                        if question in question_category['object_tracking']:
                                            question_category['object_tracking'][question].append(held_block)
                                        else:
                                            question_category['object_tracking'][question] = [held_block] 
                                        solo_sequence[valid_action_sequences[index]] = question_category
                                    elif fluent.startswith('on('):
                                        match = re.search(r'\((.*?)\)', fluent)
                                        block_above, block_below = match.group(1)[0:2], match.group(1)[3:5]
                                        question = f"Where is block {block_above}?"
                                        if question in question_category['object_tracking']:
                                            question_category['object_tracking'][question].append(block_above)
                                        else:
                                            question_category['object_tracking'][question] = [block_above] 
                                        solo_sequence[valid_action_sequences[index]] = question_category    
                                    elif fluent.startswith('handempty'):
                                        question = "Is the hand empty?"
                                        if question in question_category['object_tracking']:
                                            question_category['object_tracking'][question].append("yes")
                                        else:
                                            question_category['object_tracking'][question] = ["yes"] 
                                        solo_sequence[valid_action_sequences[index]] = question_category  
                            elif value['part_of_plan?'] == False:
                                if k == "fluents" and len(v) == 0:
                                    question = f"Is the {key} executable?"
                                    if question in question_category['executable_questions_answers']:
                                        question_category['executable_questions_answers'][question] = value['feasible?']
                                    else:
                                        question_category['executable_questions_answers'][question] = value['feasible?']    
                                    solo_sequence[valid_action_sequences[index]]= question_category     
                        elif k == "fluents" and len(v) == 0:
                            question = f"Is the {key} executable?"
                            if question in question_category['executable_questions_answers']:
                                question_category['executable_questions_answers'][question] = value['feasible?']
                            else:
                                question_category['executable_questions_answers'][question] = value['feasible?']    
                            solo_sequence[valid_action_sequences[index]]= question_category
            return solo_sequence    
            
        def generate_composite_1_q_a(self):
            # main_question_answers_list = []
            # valid_action_sequences = []
            # for item in self.data:
            #     for key, value in item.items():
            #         if value['part_of_plan?'] == True:
            #             valid_action_sequences.append(key)
            # for index, item in enumerate(self.data):
            #     main_question_answers_list.append(self.create_composite_qa_1(item,valid_action_sequences,index))
            # return main_question_answers_list
            import random
            main_datalist = []
            valid_action_sequences = []
            invalid_action_sequences = defaultdict(dict)
            composite_question_1 = defaultdict(dict)
            for x in self.data:
                for key, value in x.items():
                    if value['part_of_plan?'] == True:
                        valid_action_sequences.append(key)
            for i in range(len(self.data)-1):
                key_list = []
                for key, value in self.data[i+1].items():
                    if value['part_of_plan?'] == False and value['feasible?'] == False and len(value['fluents']) == 0:
                        random_action = random.choice(list(self.data[i+1].keys())) 
                        key_list.append(random_action)
                custom_valid_action_sequences = valid_action_sequences.copy()
                custom_valid_action_sequences.insert(i+1,key_list[0])
                question_1 = f"for the sequence {custom_valid_action_sequences}, which action causes the sequence to be inexecutable?"
                answer_1 = key_list[0]
                question_2 = f"for the sequence {custom_valid_action_sequences}, after how many timesteps does the sequence become inexecutable?"
                question_3 = f"Given the sequence {custom_valid_action_sequences}, what is the state before the first infeasible action in the sequence?"
                answer_2 = i+1
                if question_1 in invalid_action_sequences["Identifying_actions"]:
                    invalid_action_sequences["Identifying_actions"][question_1].append(answer_1)
                elif question_1 not in invalid_action_sequences["Identifying_actions"]:
                    invalid_action_sequences["Identifying_actions"][question_1] = [answer_1]
                if question_2 in invalid_action_sequences["Identifying_actions"]:
                    invalid_action_sequences["Identifying_actions"][question_2].append(answer_2)
                elif question_2 not in invalid_action_sequences["Identifying_actions"]:
                    invalid_action_sequences["Identifying_actions"][question_2] = [answer_2] 
                for key, value in self.data[i].items():
                    if value['part_of_plan?'] == True and value['fluents'] != []:
                        if question_3 in composite_question_1["Composite_question_1"]:
                            composite_question_1["Composite_question_1"][question_3].append(value['fluents'])
                        else:
                            composite_question_1["Composite_question_1"][question_3] = [value['fluents']]         
                del custom_valid_action_sequences   
                del key_list
            for index, value in enumerate(self.data):
                main_datalist.append(self.create_breakdown_composite_qa_1(value,valid_action_sequences,index)) 
            main_datalist.append(invalid_action_sequences)
            main_datalist.append(composite_question_1)    
            return main_datalist                 

In [18]:

blocksworld_instance = Blocksworld('C:/Users/shri1/reasoning_about_actions/data.jsonl')
result = blocksworld_instance.generate_composite_1_q_a()

In [19]:
result

[defaultdict(dict,
             {'action_unstack(b2,b1)': defaultdict(dict,
                          {'executable_questions_answers': {'Is the action_pick_up(b2) executable?': False,
                            'Is the action_pick_up(b1) executable?': False,
                            'Is the action_stack(b1,b2) executable?': False,
                            'Is the action_put_down(b1) executable?': False,
                            'Is the action_unstack(b1,b2) executable?': False,
                            'Is the action_unstack(b2,b1) executable?': True,
                            'Is the action_put_down(b2) executable?': False,
                            'Is the action_stack(b2,b1) executable?': False},
                           'object_tracking': {'Which blocks are on the table?': ['b1'],
                            'Which blocks are clear?': ['b1'],
                            'Which block am I holding?': ['b2']}})}),
 defaultdict(dict,
             {'action_put_down(b2